# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
C. Larkin  ->  C. Larkin  |  ['C. Larkin']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 50 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.10270


extracting tarball to tmp_2402.10270...

 done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 87 bibliographic references in tmp_2402.10270/sample631.bbl.
syntax error in line 431: '=' expected
Retrieving document from  https://arxiv.org/e-print/2402.10337


extracting tarball to tmp_2402.10337...

 done.
Retrieving document from  https://arxiv.org/e-print/2402.10431


extracting tarball to tmp_2402.10431...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.10270-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.10270) | **CLusters in the Uv as EngineS (CLUES). II. Sub-kpc scale outflows driven  by stellar feedback**  |
|| M. Sirressi, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-02-19*|
|*Comments*| *Accepted for publication on the Astronomical Journal on 14th February 2024. 32 pages, 13 figures*|
|**Abstract**| We analyze the far-ultraviolet spectroscopy of 20 young and massive star clusters (YSCs) in 11 nearby star-forming galaxies. We probe the interstellar gas intervening along the line of sight, detecting several metal absorption lines of a wide range of ionization potentials, from 6.0 eV to 77.5 eV. Multiple-component Voigt fits to the absorption lines are used to study the kinematics of the gas. We find that nearly all targets in the sample feature gas outflowing from 30 up to 190 km per second, often both in the neutral and ionized phase. The outflow velocities correlate with the underlying stellar population properties directly linked to the feedback: the mass of the YSCs, the photon production rate and the instantaneous mechanical luminosity produced by stellar winds and SNe. We detect a neutral inflow in 4 targets, which we interpret as likely not associated with the star cluster but tracing larger scale gas kinematics. A comparison between the outflows energy and that produced by the associated young stellar populations suggests an average coupling efficiency of 10 per cent with a broad scatter. Our results extend the relation found in previous works between galactic outflows and the host galaxy star-formation rate to smaller scales, pointing towards the key role that clustered star formation and feedback play in regulating galaxy growth. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.10337-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.10337) | **LoVoCCS -- II. Weak Lensing Mass Distributions, Red-Sequence Galaxy  Distributions, and Their Alignment with the Brightest Cluster Galaxy in 58  Nearby X-ray-Luminous Galaxy Clusters**  |
|| S. Fu, et al. -- incl., <mark>C. Larkin</mark> |
|*Appeared on*| *2024-02-19*|
|*Comments*| *40 pages, 16 figures, 5 tables. Submitted to ApJ. Comments are welcome and appreciated*|
|**Abstract**| The Local Volume Complete Cluster Survey (LoVoCCS) is an on-going program to observe nearly a hundred low-redshift X-ray-luminous galaxy clusters (redshifts $0.03<z<0.12$ and X-ray luminosities in the 0.1-2.4 keV band $L_{X500c}>10^{44}$ erg/s) with the Dark Energy Camera (DECam), capturing data in $u,g,r,i,z$ bands with a $5\sigma$ point source depth of approximately 25-26th AB magnitudes. Here, we map the aperture masses in 58 galaxy cluster fields using weak gravitational lensing. These clusters span a variety of dynamical states, from nearly relaxed to merging systems, and approximately half of them have not been subject to detailed weak lensing analysis before. In each cluster field, we analyze the alignment between the 2D mass distribution described by the aperture mass map, the 2D red-sequence (RS) galaxy distribution, and the brightest cluster galaxy (BCG). We find that the orientations of the BCG and the RS distribution are strongly aligned throughout the interiors of the clusters: the median misalignment angle is 19 deg within 2 Mpc. We also observe the alignment between the orientations of the RS distribution and the overall cluster mass distribution (by a median difference of 32 deg within 1 Mpc), although this is constrained by galaxy shape noise and the limitations of our cluster sample size. These types of alignment suggest long-term dynamical evolution within the clusters over cosmic timescales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.10431-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.10431) | **Spatial Variations of Dust Opacity and Grain Growth in Dark Clouds:  L1689, L1709 and L1712**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2024-02-19*|
|*Comments*| *Accepted for publication in ApJ (16 pages, 8 figures, 3 tables)*|
|**Abstract**| The far-infrared (FIR) opacity of dust in dark clouds within the Ophiuchus molecular cloud is investigated through multi-wavelength infrared observations from UKIDSS, Spitzer and Herschel. Employing the infrared color excess technique with both near-infrared (NIR) and mid-infrared (MIR) photometric data, a high-resolution extinction map in the $K$ band ($A_K$) is constructed for three dark clouds: L1689, L1709, and L1712. The derived extinction map has a resolution of $1'$ and reaches a depth of $A_K\sim3$ mag. The FIR optical depths $\tau_{250}$ at a reference wavelength of $250\,\rm \mu m$ are obtained by fitting the Herschel PACS and SPIRE continuum data at 100, 160, 250, 350 and 500 $\rm \mu m$ using a modified blackbody model. The average dust opacity per unit gas mass at $250\rm \mu m$, $r\kappa_{250}$ is determined through a pixel-by-pixel correlation of $\tau_{250}$ with $A_K$, yielding a value of approximately $0.09\,\rm cm^2\,g^{-1}$, which is about 2-3 times higher than the typical value in the diffuse interstellar medium (ISM). Additionally, an independent analysis across 16 sub-regions within the Ophiuchus cloud indicates spatial variations in dust opacity, with values ranging from 0.07-0.12$\,\rm cm^2\,g^{-1}$. Although the observed trend of increasing dust opacity with higher extinction implies grain growth, our findings indicate that rapid grain growth clearly not yet occurred in the dark clouds studied in this work. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.10270.md
    + _build/html/tmp_2402.10270/./figures/SFR_vel_relations.png
    + _build/html/tmp_2402.10270/./figures/correlations_wAge_holl.png
    + _build/html/tmp_2402.10270/./figures/voigtfit.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\HI}{\mbox{H~{\sc i}}}$
$\newcommand{\HII}{\mbox{H~{\sc ii}}}$
$\newcommand{\AlII}{\mbox{Al~{\sc ii}}}$
$\newcommand{\SiIII}{\mbox{Si~{\sc iii}}}$
$\newcommand{\SiII}{\mbox{Si~{\sc ii}}}$
$\newcommand{\SII}{\mbox{S~{\sc ii}}}$
$\newcommand{\CII}{\mbox{C~{\sc ii}}}$
$\newcommand{\CIII}{\mbox{C~{\sc iii}}}$
$\newcommand{\SiIV}{\mbox{Si~{\sc iv}}}$
$\newcommand{\CIV}{\mbox{C~{\sc iv}}}$
$\newcommand{\NII}{\mbox{N~{\sc ii}}}$
$\newcommand{\NV}{\mbox{N~{\sc v}}}$
$\newcommand{\SV}{\mbox{S~{\sc v}}}$
$\newcommand{\msun}{\hbox{M_\odot}}$</div>



<div id="title">

# CLusters in the Uv as EngineS (CLUES). II. Sub-kpc scale outflows driven by stellar feedback

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.10270-b31b1b.svg)](https://arxiv.org/abs/2402.10270)<mark>Appeared on: 2024-02-19</mark> -  _Accepted for publication on the Astronomical Journal on 14th February 2024. 32 pages, 13 figures_

</div>
<div id="authors">

M. Sirressi, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We analyze the far-ultraviolet (FUV, 1130 $\rm Å$ to 1770 $\rm Å$ restframe) spectroscopy of 20 young ( $<$ 50 Myr) and massive ( $>10^4$ $\msun$ ) star clusters (YSCs) in 11 nearby star-forming galaxies. We probe the interstellar gas intervening along the line of sight, detecting several metal absorption lines of a wide range of ionization potentials, from 6.0 eV to 77.5 eV.Multiple-component Voigt fits to the absorption lines are used to study the kinematics of the gas. We find that nearly all targets in the sample feature gas outflowing from 30 km s $^{-1}$ up to 190 km s $^{-1}$ , often both in the neutral and ionized phase. The outflow velocities correlate with the underlying stellar population properties directly linked to the feedback: the mass of the YSCs, the photon production rate and the instantaneous mechanical luminosity produced by stellar winds and SNe. We detect a neutral inflow in 4 targets, which we interpret as likely not associated with the star cluster but tracing larger scale gas kinematics. A comparison between the outflows’ energy and that produced by the associated young stellar populations suggests an average coupling efficiency of 10 \% with a broad scatter.Our results extend the relation found in previous works between galactic outflows and the host galaxy star-formation rate to smaller scales, pointing towards the key role that clustered star formation and feedback play in regulating galaxy growth.

</div>

<div id="div_fig1">

<img src="tmp_2402.10270/./figures/SFR_vel_relations.png" alt="Fig4" width="100%"/>

**Figure 4. -** Outflow velocities plotted against (from top to bottom): the SFR, the SFR per unit stellar mass and the SFR per unit area. The data points plotted are taken from three different samples: in blue and orange the CLUES neutral and ionized outflows respectively, in red the CLASSY galactic outflows \citep{Xu2022}, the empty circles represent the data points of \cite{Heckman2015}, the black squares the data points of \cite{Chisholm2015}. In the top panel we report the least-square linear regression fit for the black squares points, the blue points and the red crosses, separately. (*fig:sfr*)

</div>
<div id="div_fig2">

<img src="tmp_2402.10270/./figures/correlations_wAge_holl.png" alt="Fig12" width="100%"/>

**Figure 12. -** Outflow velocities plotted against six different stellar properties: age of young stellar population, mass of the young stellar population, mechanical luminosity of stellar winds, mechanical luminosity of SNe, photon production rate and total mechanical luminosity from stellar winds and SNe. The blue data points represent the neutral gas, whereas the orange data points represent the ionized gas. Hollow markers (NGC1566-YSC2, NGC4485-YSC2, NGC1566-YSC1, NGC7793-YSC2, NGC1313-YSC2, NGC4656-YSC1) indicate the targets with an age uncertainty larger than 25\%, which gives a sense of which data points are the most uncertain in all the plots. Error bars for the outflow velocities are plotted. $\tau$ and p are two parameters of the Kendall statistics performed to assess the presence or lack of correlation (see text). (*fig:correlations*)

</div>
<div id="div_fig3">

<img src="tmp_2402.10270/./figures/voigtfit.png" alt="Fig11" width="100%"/>

**Figure 11. -** Best-fit values of the multi-component Voigt fit analysis of the interstellar gas kinematics. The panels from left to right show the central velocity, the column density of $\SiII$$ $ and $\SiIV$$ $ and the $b$ parameter of the outflow and inflow components detected). The blue circles represent the neutral gas traced by $\SiII$, the orange stars represent the ionized gas traced by $\SiIV$. The empty blue circles mark the gas components inflowing towards the target star cluster. M51-YSC1 is the exception target for which we used $\CIV$$ $ instead of $\SiIV$. The targets are sorted by the age of the FUV dominant stellar population (*fig:voigtfit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.10270"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

372  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
